<h1> 1. Data Loading ---- start </h1>

In [8]:
#Purpose : CategoryMapping

# importing Libs And SQL connection
import pandas as pd
import numpy as np
import pymssql

server='172.30.11.143'
user='developde'
password ='everyday@123'
database='workbase'
sql_conn = pymssql.connect(server, user, password,database )
cursor = sql_conn.cursor()

print("Connection Done!")



#loading Data from files
tbStageRawdata_last3months = pd.read_csv("D:\\API_Section\\AudienceProfile\\Files\\tbStageRawdata_last3months.csv",na_values=['""', '??' , 'NULL', 'NaN', 'n/a', 'nan', 'null','None'] )   
tbActivityPagesCount = pd.read_csv("D:\\API_Section\\AudienceProfile\\Files\\tbActivityPagesCount.csv",na_values=['""', '??' , 'NULL', 'NaN', 'n/a', 'nan', 'null','None'] ) 
tbActivityPages = pd.read_csv("D:\\API_Section\\AudienceProfile\\Files\\tbActivityPages.csv",encoding = "utf-8",na_values=['""', '??' , 'NULL', 'NaN', 'n/a', 'nan', 'null','None'] ) 
tbstagedemo = pd.read_csv(
    "D:\\API_Section\\AudienceProfile\\Files\\tbstagedemo.csv",      # relative python path to subdirectory
    engine ='python',
    sep=',' ,         # Tab-separated value file.
    encoding='utf8',   # For proper encoding 
     error_bad_lines=False,
#     nrows=100,
    quotechar='"',        # single quote allowed as quote character
#     dtype={"MasterUserID": np.int32, "ZipCode":np.int32, "age":np.int32},            # Parse the salary column as an integer 
#     usecols=['MasterUserID', 'Email','FirstName','MiddleName','LastName','Address1','Address2','City','State','ZipCode','MaxDatekey','DerivedSpecialty','Specialty','Profession','Profession' ,'age','DOB'], # Only load the three columns specified.
    usecols=['MasterUserID', 'Email','Specialty'],
    #     parse_dates=['InsertedDate','MaxDatekey'],     # Intepret the birth_date column as a date
    #skiprows=10,         # Skip the first 10 rows of the file
    na_values=['""', '??' , 'NULL', 'NaN', 'n/a', 'nan', 'null','None']       # Take any '.' or '??' values as NA
)
# tbstagedemo.head(2)

tbActivityTBIDPages =pd.read_csv("D:\\API_Section\\AudienceProfile\\Files\\tbActivityTBIDPages.csv",na_values=['""', '??' , 'NULL', 'NaN', 'n/a', 'nan', 'null','None'] ) 
# tb_CategoryData =  pd.read_csv("D:\\API_Section\\AudienceProfile\\Files\\tb_CategoryData.csv",na_values=['""', '??' , 'NULL', 'NaN', 'n/a', 'nan', 'null','None'] ) 
print("Data Loaded!!")
print(tbStageRawdata_last3months.shape)
print(tbActivityPagesCount.shape)
print(tbActivityPages.shape)
print(tbstagedemo.shape)
print(tbActivityTBIDPages.shape)





Connection Done!


Skipping line 198421: ',' expected after '"'
Skipping line 203839: ',' expected after '"'
Skipping line 234487: ',' expected after '"'
Skipping line 242413: ',' expected after '"'
Skipping line 356393: ',' expected after '"'
Skipping line 357175: ',' expected after '"'
Skipping line 381112: ',' expected after '"'
Skipping line 414699: ',' expected after '"'
Skipping line 495190: ',' expected after '"'
Skipping line 511900: ',' expected after '"'


Data Loaded!!
(923191, 4)
(268094, 2)
(4096139, 9)
(513686, 3)
(2582925, 5)


In [10]:
#loading Data from SQL Server
query1 = "SELECT  * FROM workbase..vw_CategoryData(nolock)"
df1 = pd.read_sql(query1, sql_conn)
df1.head(2)
vw_CategoryData = df1.copy()
vw_CategoryData.shape

(805, 7)

In [11]:
#loading Data from SQL Server
query2 = "SELECT  * FROM workbase..vw_ArticleData(nolock)"
df2 = pd.read_sql(query2, sql_conn)
df2.head(2)
vw_ArticleData = df2.copy()
vw_ArticleData.shape

(21200, 7)

<h1> Data Loading ---- End  </h1>
<hr style="border-top: 4px dotted red;">

<h1> 2. Data  Cleaning ---- Start  </h1>

In [12]:
# data Cleaning 1 
print(tbStageRawdata_last3months.isnull().sum())
null_data = tbStageRawdata_last3months[tbStageRawdata_last3months.isnull().any(axis=1)]
print(null_data.shape)
null_data.head(2)
# filling a null values using fillna()  
tbStageRawdata_last3months["Email"].fillna("No Email", inplace = True)  
tbStageRawdata_last3months.isnull().sum()

MasterUserID      0
Source            0
CreateDTTM        0
Email           278
dtype: int64
(278, 4)


MasterUserID    0
Source          0
CreateDTTM      0
Email           0
dtype: int64

In [13]:
# data Cleaning 2
print(tbstagedemo.isnull().sum())
null_data1 = tbstagedemo[tbstagedemo.isnull().any(axis=1)]
print(null_data1.shape)
null_data1.head(2)
# filling a null values using fillna()  
tbstagedemo["Email"].fillna("No Email", inplace = True)  
tbstagedemo["Specialty"].fillna("No Specialty", inplace = True)
tbstagedemo.isnull().sum()

MasterUserID       0
Email            274
Specialty       5072
dtype: int64
(5072, 3)


MasterUserID    0
Email           0
Specialty       0
dtype: int64

In [14]:
# data Cleaning 3
tbActivityPages_copy = tbActivityPages.copy()
print(tbActivityPages.isnull().sum())
tbActivityPages['TBID'] = tbActivityPages['CleanPage'].str.extract('(/\d+)', expand=True)
null_data3 = tbActivityPages[tbActivityPages.isnull().any(axis=1)]
print(null_data3.shape)
null_data3.head(2)
# filling a null values using fillna()  
tbActivityPages["DrugName"].fillna("No DrugName", inplace = True)  
tbActivityPages["DrugListMatchkey"].fillna(0, inplace = True)  
tbActivityPages["DrugListMatchkey"].fillna(0, inplace = True)
# tbActivityPages["TBID"].fillna(0, inplace = True) 
tbActivityPages.isnull().sum()
tbActivityPages = tbActivityPages.dropna()
tbActivityPages = tbActivityPages.reset_index(drop=True)
tbActivityPages['TBID'] =tbActivityPages['TBID'].map(lambda x: x.lstrip('/'))
tbActivityPages = tbActivityPages.dropna()
tbActivityPages = tbActivityPages.reset_index(drop=True)
tbActivityPages.isnull().sum()
print(tbActivityPages.shape)

ID                        0
MasterUserID              0
Zones                     0
CleanPage                 0
ActivityType              0
Inserteddate              0
DrugName            1629067
DrugListMatchkey    1629067
IsValidBrand              0
dtype: int64
(1711317, 10)
(3952996, 10)


In [15]:
tbActivityPages.fillna(0, inplace=True)
columns = ["MasterUserID", "Zones", "CleanPage", "ActivityType", "Inserteddate", "DrugName", "DrugListMatchkey","IsValidBrand","TBID"]
tbActivityPages = tbActivityPages.reindex(columns=columns)
tbActivityPages['TBID'] = tbActivityPages['TBID'].astype(np.int64)

In [16]:
# data Cleaning 4
tbActivityTBIDPages = tbActivityTBIDPages.rename(columns={"Tbid": "TBID"})
print(tbActivityTBIDPages.isnull().sum())
null_data4 = tbActivityTBIDPages[tbActivityTBIDPages.isnull().any(axis=1)]
print(null_data4.shape)
null_data4.head(2)
# filling a null values using fillna()  
tbActivityTBIDPages["categorytitle"].fillna("NoCategory", inplace = True)  
tbActivityTBIDPages.isnull().sum()

ID                  0
MasterUserID        0
TBID                0
categorytitle    5691
Inserteddate        0
dtype: int64
(5691, 5)


ID               0
MasterUserID     0
TBID             0
categorytitle    0
Inserteddate     0
dtype: int64

In [17]:
# data Cleaning 5
print(vw_ArticleData.isnull().sum())
null_data5 = vw_ArticleData[vw_ArticleData.isnull().any(axis=1)]
print(null_data5.shape)
null_data5.head(2)
# filling a null values using fillna()  
vw_ArticleData["Title"].fillna("NoTitle", inplace = True)
vw_ArticleData["Writer"].fillna("NoWriter", inplace = True)
vw_ArticleData= vw_ArticleData.dropna(axis = 0, how ='any') 
vw_ArticleData.isnull().sum()

TBID                0
Title               4
Writer            457
CleanPageURL        0
Category            0
ParentCategory      1
ContentType         0
dtype: int64
(462, 7)


TBID              0
Title             0
Writer            0
CleanPageURL      0
Category          0
ParentCategory    0
ContentType       0
dtype: int64

In [18]:
# data Cleaning 6
print(tbActivityPagesCount.isnull().sum())
tbActivityPagesCount.shape

MasterUserID    0
PageCount       0
dtype: int64


(268094, 2)

<h1> Data Cleaning ---- End  </h1>
<hr style="border-top: 4px dotted red;">

<h1> 3. Data Merging ---- End  </h1>


In [ ]:
# print(tbStageRawdata_last3months.head(1))
# print(tbActivityPagesCount.head(1))
# print(tbActivityPages.head(1))
# print(tbstagedemo.head(1))
# print(vw_ArticleData.head(1))
# print(vw_CategoryData.head(1))

In [19]:
print(len(vw_ArticleData.ParentCategory.unique()), " & ", len(vw_ArticleData.Category.unique()))

59  &  449


In [20]:
print(tbActivityPages.shape)    # before (4096139, 9) now (3952996, 9)
tbActivityPages.head(1)

(3952996, 9)


,MasterUserID,Zones,CleanPage,ActivityType,Inserteddate,DrugName,DrugListMatchkey,IsValidBrand,TBID
0,5000062,/obgyn/pregnancy,www.medpagetoday.com/obgyn/pregnancy/80782,GA,2019-10-15,No DrugName,0.0,False,80782


In [21]:
print(vw_ArticleData.shape)  # (21179, 7)
vw_ArticleData.head(1)

(21199, 7)


,TBID,Title,Writer,CleanPageURL,Category,ParentCategory,ContentType
0,82526,Hypothermia Helps After Cardiac Arrest With No...,Crystal Phend,www.medpagetoday.com/cardiology/arrhythmias/82526,Arrhythmias,Cardiology,News Brief


In [22]:
data1=  vw_ArticleData.copy()
data2 =  tbActivityPages.copy()

In [23]:
names = vw_ArticleData.ParentCategory.unique()
names

array(['Cardiology', 'Dermatology', 'Endocrinology', 'Gastroenterology',
       'Oncology/Hematology', 'Infectious Disease', 'Neurology', 'OB/Gyn',
       'Pediatrics', 'Primary Care', 'Psychiatry',
       'Public Health & Policy', 'Pulmonology', 'Rheumatology', 'Urology',
       'Ophthalmology', 'Allergy & Immunology', 'Nephrology', 'Surgery',
       'Meeting Coverage', 'Emergency Medicine', 'HIV/AIDS',
       'Pain Management', 'Geriatrics', 'Washington-Watch', 'Surveys',
       'Opinion', 'Orthopedics', 'Radiology', 'Genetics',
       'Hospital-Based Medicine', 'Practice Management', 'Critical Care',
       'Pathology', 'Oncology', 'Transplantation', 'Anesthesiology',
       'Special Reports', 'Nursing', 'Washington Watch',
       'Building the Patient-Centered Medical Home', 'Quizzes',
       'Sports Medicine', 'Clinical Focus', 'ASCO', 'Clinical Context',
       'Mastery of Medicine', 'AGA', 'Endocrine Society',
       'Clinical Challenges', 'Clinical Connection', 'AAD',
       'A

In [24]:
#merge data
result1 = pd.merge(vw_ArticleData,
                 tbActivityPages[['TBID','MasterUserID', 'ActivityType','DrugName','IsValidBrand']],
                 on='TBID')
print(result1.shape)
result1.head(1)

(3913943, 11)


,TBID,Title,Writer,CleanPageURL,Category,ParentCategory,ContentType,MasterUserID,ActivityType,DrugName,IsValidBrand
0,81867,How Frankenstein Saved Millions of Lives,"Rohin Francis, MBBS",www.medpagetoday.com/cardiology/arrhythmias/81867,Arrhythmias,Cardiology,News Brief,5002111,GA,Feasibility test,False


<h1> Data Merging ---- End  </h1>
<hr style="border-top: 4px dotted red;">

<h1> 4.Data Manipulations ---- start  </h1>


In [25]:
pivoting = result1.pivot_table('TBID', ['MasterUserID'], 'ParentCategory')
pivoting.sort_values(by=['MasterUserID'])
pivoting_copy= pivoting.copy()
pivoting.head(1)


ParentCategory,AAD,AGA,ASCO,Allergy & Immunology,American College of Rheumatology,Anesthesiology,Building the Patient-Centered Medical Home,Cardiology,Case Studies,Clinical Challenges,...,Rheumatology,Special Reports,Sports Medicine,Surgery,Surveys,TedMed,Transplantation,Urology,Washington Watch,Washington-Watch
MasterUserID,,,,,,,,,,,,,,,,,,,,,
6362,NaN,NaN,NaN,NaN,NaN,NaN,NaN,81442.333333,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
pivoting = pivoting.fillna(0)
pivoting.head(1)

ParentCategory,AAD,AGA,ASCO,Allergy & Immunology,American College of Rheumatology,Anesthesiology,Building the Patient-Centered Medical Home,Cardiology,Case Studies,Clinical Challenges,...,Rheumatology,Special Reports,Sports Medicine,Surgery,Surveys,TedMed,Transplantation,Urology,Washington Watch,Washington-Watch
MasterUserID,,,,,,,,,,,,,,,,,,,,,
6362,0.0,0.0,0.0,0.0,0.0,0.0,0.0,81442.333333,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
pivoting=pivoting.replace(0.0,False)
pivoting[pivoting >= 1] = True
pivoting = pivoting * 1
pivoting.sort_values(by=['MasterUserID'])
pivoting.head(20)




ParentCategory,AAD,AGA,ASCO,Allergy & Immunology,American College of Rheumatology,Anesthesiology,Building the Patient-Centered Medical Home,Cardiology,Case Studies,Clinical Challenges,...,Rheumatology,Special Reports,Sports Medicine,Surgery,Surveys,TedMed,Transplantation,Urology,Washington Watch,Washington-Watch
MasterUserID,,,,,,,,,,,,,,,,,,,,,
6362,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
20534,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
24070,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29122,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
29900,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30600,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
37193,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
37996,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
43227,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<h1> Data Manipulations ---- End  </h1>
<hr style="border-top: 4px dotted red;">

In [28]:
# x = [(pivoting > 0).any(1)]
pivoting.loc[pivoting['Urology']==1]

ParentCategory,AAD,AGA,ASCO,Allergy & Immunology,American College of Rheumatology,Anesthesiology,Building the Patient-Centered Medical Home,Cardiology,Case Studies,Clinical Challenges,...,Rheumatology,Special Reports,Sports Medicine,Surgery,Surveys,TedMed,Transplantation,Urology,Washington Watch,Washington-Watch
MasterUserID,,,,,,,,,,,,,,,,,,,,,
5000252,0,0,1,1,0,0,1,1,0,0,...,1,0,0,1,0,0,1,1,0,0
5000840,0,0,0,1,0,0,0,1,0,0,...,1,0,0,1,0,1,0,1,0,0
5000910,0,0,1,1,1,0,1,1,1,0,...,1,1,0,0,1,1,0,1,1,1
5000926,0,0,0,0,0,0,1,1,1,0,...,0,1,0,1,0,0,0,1,0,0
5001239,0,0,0,1,0,0,1,1,0,0,...,1,0,0,1,0,1,1,1,0,0
5001957,0,1,0,0,0,0,0,1,1,0,...,1,0,0,0,0,0,1,1,0,0
5001993,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
5004611,0,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,1,0,0
5006520,0,0,0,1,0,0,1,1,1,0,...,1,1,0,0,0,0,0,1,1,0


In [97]:
# pivoting_melt = pd.melt(pivoting, id_vars=['MasterUserID'], value_vars=[1:])
# pivoting_melt

In [96]:
pivoting.columns.to_list()

['AAD',
 'AGA',
 'ASCO',
 'Allergy & Immunology',
 'American College of Rheumatology',
 'Anesthesiology',
 'Building the Patient-Centered Medical Home',
 'Cardiology',
 'Case Studies',
 'Clinical Challenges',
 'Clinical Connection',
 'Clinical Context',
 'Clinical Focus',
 'Critical Care',
 'Dermatology',
 'Emergency Medicine',
 'Endocrine Society',
 'Endocrinology',
 'Gastroenterology',
 'Genetics',
 'Geriatrics',
 'HIV/AIDS',
 'Hospital-Based Medicine',
 'IDSA',
 'Infectious Disease',
 'Innovations in Medicine',
 'Mastery of Medicine',
 'Meeting Coverage',
 'Nephrology',
 'Neurology',
 'Nursing',
 'OB/Gyn',
 'Oncology',
 'Oncology/Hematology',
 'Ophthalmology',
 'Opinion',
 'Orthopedics',
 'Pain Management',
 'Pathology',
 'Pediatrics',
 'Podcasts',
 'Practice Management',
 'Primary Care',
 'Psychiatry',
 'Public Health & Policy',
 'Pulmonology',
 'Quizzes',
 'Radiology',
 'Recent Developments in',
 'Rheumatology',
 'Special Reports',
 'Sports Medicine',
 'Surgery',
 'Surveys',
 'Ted

In [ ]:
import sklearn as sk
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from sklearn.manifold import LocallyLinearEmbedding
from sklearn.decomposition import PCA
from sklearn.decomposition import IncrementalPCA
from sklearn.decomposition import KernelPCA
from sklearn.decomposition import SparsePCA
from sklearn.manifold import MDS
from sklearn.manifold import Isomap
from sklearn.manifold import TSNE
from sklearn.decomposition import TruncatedSVD
from sklearn.random_projection import GaussianRandomProjection
from sklearn.decomposition import FastICA
from sklearn.decomposition import MiniBatchDictionaryLearning
from sklearn.random_projection import SparseRandomProjection
import keras
from keras.models import Sequential, Model
from keras.layers import Dense
from keras.optimizers import Adam

In [104]:
pivoting.shape

(256122, 59)

In [103]:
pivoting.isnull().sum()

ParentCategory
AAD                                           249975
AGA                                           251754
ASCO                                          237800
Allergy & Immunology                          250126
American College of Rheumatology              247005
Anesthesiology                                255817
Building the Patient-Centered Medical Home    223312
Cardiology                                    191789
Case Studies                                  236640
Clinical Challenges                           254248
Clinical Connection                           242354
Clinical Context                              256119
Clinical Focus                                256018
Critical Care                                 248439
Dermatology                                   245412
Emergency Medicine                            236208
Endocrine Society                             250293
Endocrinology                                 205103
Gastroenterology               

In [9]:
# pca = PCA(n_components=784, whiten = False, random_state = 2019)
# X_pca = pca.fit_transform(pivoting.values)
# X_pca_reconst = pca.inverse_transform(X_pca)

<h1> 5.Dimension Reduction ---- start  </h1>

In [61]:
# pivoting.columns.to_list()
# # pivoting.index.values
# pivoting_copy = pivoting.copy()
# pivoting['MasterUserID'] = pivoting.index
# pivoting['MasterUserID']

In [62]:
from sklearn.preprocessing import StandardScaler
features = pivoting.columns.to_list()
# Separating out the features
x = pivoting.loc[:, features].values   # ALL 59-catgories
# Separating out the target
y = pivoting.index.values   # MUIDS
# Standardizing the features
x = StandardScaler().fit_transform(x)

C:\Users\skondi\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\skondi\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [52]:
x

array([[-0.15681345, -0.13172043, -0.27757511, ..., -0.08405809,
        -0.1098176 , -0.18989129],
       [-0.15681345, -0.13172043, -0.27757511, ..., -0.08405809,
        -0.1098176 , -0.18989129],
       [-0.15681345, -0.13172043, -0.27757511, ..., -0.08405809,
        -0.1098176 , -0.18989129],
       ...,
       [-0.15681345, -0.13172043, -0.27757511, ..., -0.08405809,
        -0.1098176 , -0.18989129],
       [-0.15681345, -0.13172043, -0.27757511, ..., -0.08405809,
        -0.1098176 , -0.18989129],
       [-0.15681345, -0.13172043,  3.60262854, ..., -0.08405809,
        -0.1098176 , -0.18989129]])

In [53]:
from sklearn.decomposition import PCA 
pca = PCA(n_components=15)  # choosing 15 component, but not sure about those names?
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents)


In [55]:
# finalDf = pd.concat([principalDf, pivoting[['MasterUserID']]], axis = 1)

In [60]:
principalDf.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,-1.952616,0.374568,-0.189932,0.086888,0.321442,-0.135917,-0.086436,0.086444,-0.093492,0.261651,0.224115,-0.366212,0.076292,-0.099664,-0.096222
1,-1.138338,-0.364549,-0.207355,0.176113,0.448373,-0.167748,-0.184357,-0.340132,0.054960,-0.133467,-0.049246,-0.274530,0.401061,0.011300,-0.209780
2,-0.773990,0.045472,-1.853079,-0.215102,1.290358,-0.722060,0.947629,-0.681474,-0.646744,0.936847,0.525583,-0.195955,-0.257067,-0.897909,-0.248492
3,-1.952616,0.374568,-0.189932,0.086888,0.321442,-0.135917,-0.086436,0.086444,-0.093492,0.261651,0.224115,-0.366212,0.076292,-0.099664,-0.096222
4,-1.887391,0.185675,0.155881,0.411052,0.132032,-0.153836,0.017525,0.047663,-0.008448,-0.059283,-0.108569,0.078399,-0.187173,-0.101304,-0.035041
